# 如何使用mimic计算年龄和简单的筛选过程

## 1、连接数据库

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

from IPython.display import display,HTML
%matplotlib inline
plt.style.use('ggplot')

#创建数据库连接
sqluser='postgres'
dbname='mimic'
schema_name='mimiciii'
con=psycopg2.connect(dbname=dbname,user=sqluser)
query_schema='set search_path to '+schema_name+';'

1）这里是python链接数据库的过程，<br>
2）这里有一个关键点就是schema_name,直接指定了schema为mimiciii，也就避免了程序里每次都要在表前面加mimiciii<br>
3）也是使用psycopg2

## 2、查询

In [2]:
query=query_schema+"""
SELECT subject_id,hadm_id,icustay_id
FROM icustays
LIMIT 10
"""
df=pd.read_sql_query(query,con)
df.head()

,subject_id,hadm_id,icustay_id
0,268,110404,280836
1,269,106296,206613
2,270,188028,220345
3,271,173727,249196
4,272,164716,210407


1)执行查询语句用的是pandas包<br>
2)每次执行sql语句，都要设置schema名称

## 3、对时间进行处理

In [3]:
query=query_schema+"""
SELECT subject_id,hadm_id,icustay_id
,outtime-intime as icu_length_of_stay_interval
,EXTRACT(EPOCH FROM outtime - intime) as icu_length_of_stay
FROM icustays
LIMIT 10
"""
df=pd.read_sql_query(query,con)
df.head()

,subject_id,hadm_id,icustay_id,icu_length_of_stay_interval,icu_length_of_stay
0,268,110404,280836,3 days 05:58:33,280713.0
1,269,106296,206613,3 days 06:41:28,283288.0
2,270,188028,220345,2 days 21:27:09,250029.0
3,271,173727,249196,2 days 01:26:22,177982.0
4,272,164716,210407,1 days 14:53:09,139989.0


In [4]:
query=query_schema+"""
SELECT subject_id,hadm_id,icustay_id
,EXTRACT(EPOCH FROM outtime-intime)/60.0/60.0/24.0 as icu_stay_of_stay
FROM icustays
LIMIT 10
"""
df=pd.read_sql_query(query,con)
df.head()

,subject_id,hadm_id,icustay_id,icu_stay_of_stay
0,268,110404,280836,3.248993
1,269,106296,206613,3.278796
2,270,188028,220345,2.893854
3,271,173727,249196,2.059977
4,272,164716,210407,1.620243


要注意这里的两个方面：
1）在select 中 两个列可以直接运算，outtime-intime
2）关于时间的处理 使用了extract函数，使用epoch关键字，返回的是秒，是数值类型，再做相应运算就可以得到自己需要的数据，非常方便。

## 4、加入一些限制条件

In [5]:
query=query_schema+"""
WITH co AS
(
SELECT subject_id,hadm_id,icustay_id
,EXTRACT(EPOCH FROM outtime-intime)/60/60/24 as icu_length_of_stay
FROM icustays
LIMIT 10
)
SELECT 
    co.subject_id,co.hadm_id,co.icustay_id,co.icu_length_of_stay
FROM co
WHERE icu_length_of_stay>=2
"""
df=pd.read_sql_query(query,con)
df

,subject_id,hadm_id,icustay_id,icu_length_of_stay
0,268,110404,280836,3.248993
1,269,106296,206613,3.278796
2,270,188028,220345,2.893854
3,271,173727,249196,2.059977
4,274,130546,254851,8.814259
5,275,129886,219649,7.131412


In [7]:
query=query_schema+"""
with co as 
(
select subject_id,hadm_id,icustay_id
,extract(epoch from outtime-intime)/60/60/24 as icu_length_of_stay
from icustays
limit 10
)
select 
co.subject_id,co.hadm_id,co.icustay_id,co.icu_length_of_stay
,case
    when co.icu_length_of_stay<2 then 1
 else 0 end
     as exclusion_los
from co
"""
df=pd.read_sql_query(query,con)
df

,subject_id,hadm_id,icustay_id,icu_length_of_stay,exclusion_los
0,268,110404,280836,3.248993,0
1,269,106296,206613,3.278796,0
2,270,188028,220345,2.893854,0
3,271,173727,249196,2.059977,0
4,272,164716,210407,1.620243,1
5,273,158689,241507,1.486181,1
6,274,130546,254851,8.814259,0
7,275,129886,219649,7.131412,0
8,276,135156,206327,1.337836,1
9,277,171601,272866,0.731273,1


1)这里有个嵌套查询，with as 首先进行查询，并将数据保存在一个临时表中，之后直接使用<br>
2)可以在列中直接使用判断语句，case else end操作，这个十分有用<br>
3)在case中还是用了when then

接下来，我们要限制患者的年龄信息，继续筛选病人

In [9]:
query=query_schema+"""
with co as
(
select icu.subject_id,icu.hadm_id,icu.icustay_id
,extract(epoch from outtime-intime)/60/60/24 as icu_length_of_stay
,icu.intime-pat.dob as age
from icustays icu
inner join patients pat
on icu.subject_id=pat.subject_id
limit 10
)

select co.subject_id,co.hadm_id,co.icustay_id,co.age
,case 
    when co.icu_length_of_stay<2 then 1
 else 0 end
 as exclusion_los
from co
"""
df=pd.read_sql_query(query,con)
df

,subject_id,hadm_id,icustay_id,age,exclusion_los
0,2,163353,243653,0 days 21:20:07,1
1,3,145834,211552,27950 days 19:10:11,0
2,4,185777,294638,17475 days 00:29:31,1
3,5,178980,214757,0 days 06:04:24,1
4,6,107064,228232,24084 days 21:30:54,0
5,7,118037,278444,0 days 15:35:29,1
6,7,118037,236754,2 days 03:26:01,1
7,8,159514,262299,0 days 12:36:10,1
8,9,150750,220597,15263 days 13:07:02,0
9,10,184167,288409,0 days 11:39:05,0


In [10]:
query=query_schema+"""
with co as
(
select icu.subject_id,icu.hadm_id,icu.icustay_id
,extract(epoch from outtime-intime)/60/60/24 as icu_length_of_stay
,icu.intime-pat.dob as age
from icustays icu
inner join patients pat
on icu.subject_id=pat.subject_id
limit 10
)

select co.subject_id,co.hadm_id,co.icustay_id,co.age
,extract(epoch from co.age)/60/60/24/365.242 as age_extract_epoch
,case 
    when co.icu_length_of_stay<2 then 1
 else 0 end
 as exclusion_los
from co
"""
df=pd.read_sql_query(query,con)
df

,subject_id,hadm_id,icustay_id,age,age_extract_epoch,exclusion_los
0,2,163353,243653,0 days 21:20:07,0.002434,1
1,3,145834,211552,27950 days 19:10:11,76.526792,0
2,4,185777,294638,17475 days 00:29:31,47.845047,1
3,5,178980,214757,0 days 06:04:24,0.000693,1
4,6,107064,228232,24084 days 21:30:54,65.942297,0
5,7,118037,278444,0 days 15:35:29,0.001779,1
6,7,118037,236754,2 days 03:26:01,0.005868,1
7,8,159514,262299,0 days 12:36:10,0.001438,1
8,9,150750,220597,15263 days 13:07:02,41.790228,0
9,10,184167,288409,0 days 11:39:05,0.001329,0


现在筛选年龄>16岁的，也就是成年患者

In [12]:
query=query_schema+"""
with co as
(
select icu.subject_id,icu.hadm_id,icu.icustay_id
,extract(epoch from outtime-intime)/60/60/24 as icu_length_of_stay
,extract(epoch from icu.intime-pat.dob)/60/60/24/365.242 as age
from icustays icu
inner join patients pat
on icu.subject_id=pat.subject_id
limit 10
)

select co.subject_id,co.hadm_id,co.icustay_id,co.age
,case 
    when co.icu_length_of_stay<2 then 1
 else 0 end
 as exclusion_los
 ,case 
     when co.age<16 then 1
  else 0 end
  as exclusion_age
from co
"""
df=pd.read_sql_query(query,con)
df

,subject_id,hadm_id,icustay_id,age,exclusion_los,exclusion_age
0,2,163353,243653,0.002434,1,1
1,3,145834,211552,76.526792,0,0
2,4,185777,294638,47.845047,1,0
3,5,178980,214757,0.000693,1,1
4,6,107064,228232,65.942297,0,0
5,7,118037,278444,0.001779,1,1
6,7,118037,236754,0.005868,1,1
7,8,159514,262299,0.001438,1,1
8,9,150750,220597,41.790228,0,0
9,10,184167,288409,0.001329,0,1


如何标记这些地儿次进入ICU的患者，这里使用到了窗口函数和Rank函数，功能十分强大 ，惊叹！

In [5]:
query=query_schema+"""
with co as
(
select icu.subject_id,icu.hadm_id,icu.icustay_id
,extract(epoch from outtime-intime)/60/60/24 as icu_length_of_stay
,extract(epoch from icu.intime-pat.dob)/60/60/24/365.242 as age
,rank() over (partition by icu.subject_id order by icu.intime ) as icustay_id_order
from icustays icu
inner join patients pat
on icu.subject_id=pat.subject_id
limit 10
)

select co.subject_id,co.hadm_id,co.icustay_id,co.age
,case 
    when co.icu_length_of_stay<2 then 1
 else 0 end
 as exclusion_los
 ,case 
     when co.age<16 then 1
  else 0 end
  as exclusion_age
from co
"""
df=pd.read_sql_query(query,con)
df

,subject_id,hadm_id,icustay_id,age,exclusion_los,exclusion_age
0,2,163353,243653,0.002434,1,1
1,3,145834,211552,76.526792,0,0
2,4,185777,294638,47.845047,1,0
3,5,178980,214757,0.000693,1,1
4,6,107064,228232,65.942297,0,0
5,7,118037,278444,0.001779,1,1
6,7,118037,236754,0.005868,1,1
7,8,159514,262299,0.001438,1,1
8,9,150750,220597,41.790228,0,0
9,10,184167,288409,0.001329,0,1
